This is an example from Simpy webpage.

In [1]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""

import random

import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, f'Customer{i:02d}', counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print(f'{arrive:7.4f} {name}: Here I am')

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print(f'{env.now:7.4f} {name}: Waited {wait:6.3f}')

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print(f'{env.now:7.4f} {name}: Finished')

        else:
            # We reneged
            print(f'{env.now:7.4f} {name}: RENEGED after {wait:6.3f}')


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

ModuleNotFoundError: No module named 'simpy'

The following is from ChatGPT.

In [3]:
import simpy
import random

# Parameters
l = 1  # Rate of customer arrivals (lambda)
u = 0.8  # Rate of service (mu)
m = 2  # Number of servers
sim_duration = 100  # Simulation duration (in time units)

# Lists to store results
waiting_times = []
system_times = []

# Customer class to model the customers arriving at the system
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time
        self.wait_time = 0
        self.service_time = 0
        self.system_time = 0

def customer_arrivals(env, servers):
    """Handles the arrival of customers."""
    customer_id = 0
    while True:
        customer_id += 1
        arrival_time = env.now
        customer = Customer(arrival_time)
        print(f"Time {env.now:.2f}: Customer {customer_id} arrives.")
        env.process(customer_service(env, customer, servers, customer_id))
        yield env.timeout(random.expovariate(1/l))  # Exponential inter-arrival time

def customer_service(env, customer, servers, customer_id):
    """Simulates the service of a customer."""
    # Request a server
    with servers.request() as request:
        print(f"Time {env.now:.2f}: Customer {customer_id} waits for a server.")
        yield request  # Wait for a free server
        
        # Customer starts service
        wait_time = env.now - customer.arrival_time
        customer.wait_time = wait_time
        service_time = random.expovariate(1/u)  # Exponential service time
        customer.service_time = service_time
        print(f"Time {env.now:.2f}: Customer {customer_id} starts service. Wait time = {wait_time:.2f}, Service time = {service_time:.2f}")

        # Simulate service time
        yield env.timeout(service_time)
        
        # Customer completes service
        customer.system_time = env.now - customer.arrival_time
        print(f"Time {env.now:.2f}: Customer {customer_id} finishes service. Total time in system = {customer.system_time:.2f}")

        # Store results
        waiting_times.append(customer.wait_time)
        system_times.append(customer.system_time)

# Initialize the environment and server pool
env = simpy.Environment()
servers = simpy.Resource(env, capacity=m)

# Start the arrival process
env.process(customer_arrivals(env, servers))

# Run the simulation
env.run(until=sim_duration)

# Output the results
print(f"\nAverage waiting time: {sum(waiting_times) / len(waiting_times):.2f}")
print(f"Average system time: {sum(system_times) / len(system_times):.2f}")
print(f"Total customers served: {len(waiting_times)}")

ModuleNotFoundError: No module named 'simpy'